### Context

In this notebook, I'll use the data that was cleaned previously and will use gensim models to identify common topics.

In [ ]:
import pandas as pd
import nltk

In [ ]:
import gensim
import gensim.corpora as corpora
import re
import spacy
from gensim.utils import simple_preprocess

In [ ]:
instagram = pd.read_csv("/Users/ana/ironhack_coding/projects/instagram-topic-prediction/datasets/instagram.csv")


## Topic Analysis - experiments

In [ ]:
# inspired here https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

Topic modeling is the process of identifying topics in a set of documents.Latent Dirichlet Allocation (LDA) is a probabilistic method for Topic Modelling.

In [ ]:

data = instagram.post.tolist()
data[:10]



In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words])

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# define stopwords

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
# term frequency
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
lda_model.print_topics()


Topics still too "dispersed". Will use other paremeters to find more "solid" topics

## Fine tuning

In [ ]:
# Build LDA model
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model2.print_topics()


In [ ]:
# tuning trying different passes, etc

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics()

## Nouns only

In [ ]:
# keeping nouns only

from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)
            

In [ ]:
data_nouns = instagram["post"].apply(str).apply(nouns)
data_nouns[:10]

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_nouns)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics()


#### Topics if only accounting for nouns (above)
To be updated. topics below before adding wellness hashtags
- food, female topics (period, estrogen)
- woman, leadership
- recipe?
- disease, medical procedures
- symptom/pain


In [ ]:
# reducing chunksize

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics()


### Topics when chunkside is reduced (above)
To be updated. topics below before adding wellness hashtags
- procedure, treatment
- woman, life
- fruit, plann
- disease, symptom
- day, food


In [ ]:
# chunkside back to 100, passes 50, 4 topics

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics()



### with 4 topics
To be updated. topics below before adding wellness hashtags
- woman, food, weight, ageing
- day, year, goal - probably resolutions
- ?
- disease, procedure

In [ ]:
# 3 topics
lda_model3 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
lda_model3.print_topics()


### 3 topics (above)
To be updated. topics below before adding wellness hashtags
- diet/food: water, recipe, protein
- resolution: the healthy life, decisions, woman
- diseases: fibroid, woman, procedure, treatment, pain, opinion



### Model Perplexity and Coherence Score

In [ ]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model3, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# coherence score is anything between 0 and 1. 1 being good.


### Visualisation

In [ ]:
# libraries
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model3, corpus, id2word)
vis

### Conclusion
Afer experimenting with different parameters, 3 topics seems to give us a solid idea of the different existent topics founds in the instagram posts. 
This might need to be changed/ further "tuned" if more data becomes available.


In [ ]:
type(lda_model3)

### Bulding LDA Mallet Model

LDA Mallet Model often gives a better quality of topics

In [ ]:
from gensim.models import CoherenceModel

mallet_path = '/Users/ana/Downloads/mallet-2.0.8/bin/mallet' 

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

### compute Mallet Model with 9 topics 

In [ ]:
ldamallet_9 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=9, id2word=id2word)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet_9, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score for 9 topics: ', coherence_ldamallet)

### with 12 topics

In [ ]:
ldamallet_12 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, id2word=id2word)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet_12, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

### 15 topics

In [ ]:
ldamallet_15 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet_15, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

### 20 topics

In [ ]:
ldamallet_20 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet_20, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

15 topics seems to be the best option here

In [ ]:
# Select the model and print the topics
optimal_model = ldamallet_15
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=8))

In [ ]:
# adding keywrods to dataframe 

In [ ]:
def format_topics_sentences(ldamodel=ldamallet_15, corpus=corpus, texts=data_nouns):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# add date to the above

In [ ]:
df_dominant_topic.shape

In [ ]:
instagram.shape

In [ ]:
df_dominant_topic['Date'] = instagram['timestamp'].values

In [ ]:
instagram_topics = df_dominant_topic

In [ ]:
instagram_topics.head()

In [ ]:
#doc to csv

In [ ]:
instagram_topics.to_csv("/Users/ana/ironhack_coding/projects/final_project/final-project/datasets/instagram_topics.csv")

In [ ]:
# understanding more about the topics

In [ ]:
pd.set_option('display.max_colwidth', -1)
#pd.options.display.max_rows
pd.set_option('display.max_rows', None)

In [ ]:
instagram_topics[instagram_topics.Dominant_Topic == 1.0]

In [ ]:
instagram_topics["Dominant_Topic"].value_counts()

#### Topic Interpretation
- 0 - benefits of certain foods
- 1 - inspirational, live, love, etc
- 2- friend, family
- 3- skin
- 4- women diseases
- 5- energy, mind stress, 
- 6 - calories, meal, protein
- 7 - daily routines (morning, night routines)
- 8 - weight program
- 9 - day , today
- 10 - Undefined
- 11 - recipe
- 12 - change habits
- 13 - living the moment
- 14 - support, team, mental health?
